In [2]:
## Run ML methods on PanPred and panta outputs 
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import datasets
from sklearn import svm
import random
import os
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from numpy import genfromtxt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
import pandas as pd
import numpy as np

In [3]:
def run_ML(X, y, data_set, approach="Default", feature_selection = False, FS_method = 'mutual_info_classif', X2 = None):
    # base_dir = '/data/hoan/amromics/prediction/output/predPantaPanPred'+version
    base_dir = '/data/hoan/amromics/prediction/output/predPantaPanPred_v6'
    if not os.path.isdir(base_dir):
        os.system('mkdir '+ base_dir)
    score = []
    methods = []
    n_loops = 2
    n_folds = 5
    n_samples = y.shape[0]
    if X2 is not None:
        print("Original shape of input:", X.shape, X2.shape)
    for i in range(n_loops):
        cv = KFold(n_splits=n_folds, shuffle=True, random_state = i)
        for fold, (train_idx, test_idx) in enumerate(cv.split(X)):
            path_dir = base_dir +'/' + data_set + '_run_'+str(i)+'_'+ 'fold_'+str(fold)+'_'+approach
            print('Run: ', i, ', fold: ', fold)
            X_train = X[train_idx]
            X_test = X[test_idx]
            y_train = y[train_idx]
            y_test = y[test_idx]
            if False:
                if i <= 0:
                    print("Run feature selection", 'method = ', FS_method)
                if FS_method == 'mutual_info_classif':
                    fs_fit = SelectKBest(mutual_info_classif, k=1000).fit(X_train, y_train)
                elif FS_method == 'chi2':
                    fs_fit = SelectKBest(chi2, k=1000).fit(X_train, y_train)
                else:
                    print("Please input correct feature selection method")
                X_train = fs_fit.transform(X_train)
                X_test = fs_fit.transform(X_test)
            if X2 is not None:
                X2_train = X2[train_idx]
                X2_test = X2[test_idx]
                if feature_selection:
                    fs2_fit = SelectKBest(chi2, k=20000).fit(X2_train, y_train)
                    X2_train = fs2_fit.transform(X2_train)
                    X2_test = fs2_fit.transform(X2_test)
                X_train = np.append(X_train, X2_train, axis = 1)
                X_test = np.append(X_test, X2_test, axis = 1)
              
            # print("Standize the data")
            # Save the test true labels
            np.savetxt(path_dir + "_test_true_labels.csv", y_test, delimiter=",")
            if i <= 0 and fold <= 0:
                print("n_samples: ", n_samples)
                print("Reduced shape of the data: ", X_train.shape, X_test.shape)
            print(test_idx[:10])
            model = lgb.LGBMClassifier()
            model.fit(X_train, y_train)
            methods.append('LightGBM')
            print(methods[-1], end =', ')
            # clfG.fit(X_train, y_train)
            y_predict=model.predict(X_test) 
            np.savetxt(path_dir + "_LightGBM_labels.csv", y_predict, delimiter=",")
            score.append(f1_score(y_predict, y_test, average='macro'))
        
    # Print statistics
    n_methods = len(set(methods))
    score_np = np.array(score)
    # Each column is a method
    print(methods[:n_methods])
    average_score = np.mean(score_np.reshape((n_loops*n_folds, n_methods)), axis=0)
    print(np.round(average_score, 2))

In [4]:
metadata_panta = pd.read_csv("/data/hoan/amromics/prediction/data/Ecoli1936metafiles/metadata_final.csv")

In [5]:
# np.unique(metadata_panta['Year'])

In [6]:
snp_mat = np.load('/data/hoan/amromics/prediction/data/kmer_full_mat_VT1_DNA.npy') # pantaDifferSite

In [7]:
snp_mat.shape

(1653, 287260)

In [22]:
# https://stackoverflow.com/questions/41458834/how-is-scikit-learn-cross-val-predict-accuracy-score-calculated
## No _ in the method name, please
from pangraph.utils import binary_label
max_idx_amr = 14; # max value = 14

In [ ]:
# for idx in range(2, 3):
for idx in range(2, max_idx_amr):
    y_class = metadata_panta.iloc[:,idx].values
    print(metadata_panta.columns[idx])
    # y = np.array([1 if y_class[i]=='R' else 0 for i in range(len(y_class))]) version _v5
    y, nonenan_index = binary_label(y_class) # v6
    # pa_matrix_new = pa_matrix[nonenan_index, ]
    y_new = y[nonenan_index]
    snp_mat_new = snp_mat[nonenan_index,]
    # Run unimodal gene
    # run_ML(pa_matrix, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaPangenome', False, 'mutual_info_classif', None)
    # run_ML(full_matrix, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaPangenome', False, 'mutual_info_classif', None)
    # run_ML(snp_mat, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaSnp', True, 'chi2')
    # run_ML(pa_matrix, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaCombine', False, 'mutual_info_classif', snp_mat)
    # run_ML(pa_matrix, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaCombineScale', True, 'chi2', snp_mat)
    # run_ML(pa_matrix, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaCombinehighGene', False, 'chi2', snp_mat)
    # run_ML(pa_matrix_new, y_new, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaPangenome', False, 'chi2', None)
    # run_ML(snp_mat_new, y_new, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaVT10', False, 'chi2', None)
    # run_ML(snp_mat_new, y_new, 'Ecoli1936'+'_'+metadata_panta.columns[idx], panta_single, False, 'chi2', None)
    run_ML(snp_mat_new, y_new, 'Ecoli1936'+'_'+metadata_panta.columns[idx], "KmerK31DNA", False, 'chi2', None)
    # run_ML(pa_matrix_new, y_new, 'Ecoli1936'+'_'+metadata_panta.columns[idx], panta_combine, False, 'chi2', snp_mat_new)